In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [3]:
import torchvision

In [4]:
input_path = "lisat_gaze_data_v1/"

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(), #for parallel computations
        normalize
    ]),
}

#train_subset_size = int(0.4 * len(datasets.ImageFolder(input_path + 'train_1250', transform=data_transforms['train']))) #testing hashelha b3den

image_datasets = {
    'train':
    datasets.ImageFolder(input_path + 'lisat_gaze_data_v1/train', data_transforms['train']),
    'validation':
    datasets.ImageFolder(input_path + 'lisat_gaze_data_v1/val', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=2),  # for now 2
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=2)  # for Kaggle
}

In [6]:
## For regaining the original Classes
## Mainly For Testing 
# for images, labels in dataloaders['validation']:
#     print(labels)
#     names = image_datasets['validation'].classes
#     print([names[label] for label in labels])
    
classes_labels = ['Eyes Closed','Forward','Left Mirror','Radio','Rearview','Right Mirror','Shoulder','Speedometer']


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [8]:
import torch
print(torch.version.cuda)

11.8


In [9]:
model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.BatchNorm1d(512),  # Batch normalization layer
    nn.ReLU(inplace=True),  # activation function
    nn.Linear(512, 8),
    nn.Softmax(dim=1)
)

C:\Users\AS-GP\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\AS-GP\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=3):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_accuracy = 0

    def early_stop(self, current_accuracy):
        if (current_accuracy > self.best_accuracy + self.min_delta):
            self.best_accuracy = current_accuracy
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [11]:
early_stopper = EarlyStopper(15,3)
weights_path = 'models/pytorch/Pytorch_trial_weights.pt'
def train_model(model, criterion, learning_rate,weight_decay, num_epochs=5):
    training_accuracies = []
    validation_accuracies = []
    training_accuracies.append(0)
    validation_accuracies.append(0)
    optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate, weight_decay=weight_decay)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train().to(device) #added to device
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            if phase == 'train':
                training_accuracies.append(epoch_acc.item()*100)
            else:
                validation_accuracies.append(epoch_acc.item()*100)
                if early_stopper.early_stop(epoch_acc.item()*100):
                    return model,training_accuracies,validation_accuracies
            
                if epoch_acc.item()*100 >= early_stopper.best_accuracy:
                    early_stopper.best_accuracy = epoch_acc.item()*100
                    torch.save(model.state_dict(), weights_path)
                    print(f"Model saved as it achieved the best validation accuracy so far {early_stopper.best_accuracy}" )

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model,training_accuracies,validation_accuracies 

In [12]:
criterion = nn.CrossEntropyLoss()
model_trained,training_accuracies,validation_accuracies = train_model(model, criterion,0.001,0.0001,num_epochs=100) #1 epoch for testing gpu

Epoch 1/100
----------
train loss: 1.5981, acc: 0.6912
Model saved as it achieved the best validation accuracy so far 61.53048527581916
validation loss: 1.6539, acc: 0.6153
Epoch 2/100
----------
train loss: 1.5082, acc: 0.7773
Model saved as it achieved the best validation accuracy so far 69.09995852343425
validation loss: 1.5874, acc: 0.6910
Epoch 3/100
----------
train loss: 1.4855, acc: 0.7988
validation loss: 1.6313, acc: 0.6421
Epoch 4/100
----------
train loss: 1.4755, acc: 0.8074
validation loss: 1.6424, acc: 0.6350
Epoch 5/100
----------
train loss: 1.4605, acc: 0.8227
Model saved as it achieved the best validation accuracy so far 70.65532973869763
validation loss: 1.5702, acc: 0.7066
Epoch 6/100
----------
train loss: 1.4625, acc: 0.8183
validation loss: 1.5862, acc: 0.6916
Epoch 7/100
----------
train loss: 1.4566, acc: 0.8261
validation loss: 1.6357, acc: 0.6325
Epoch 8/100
----------
train loss: 1.4510, acc: 0.8327
validation loss: 1.6251, acc: 0.6512
Epoch 9/100
---------

In [15]:
# def Plot_Training_Validation_Accuracies(epochs,training_accuracies, validation_accuracies): 
#     fig = plt.figure(figsize=(19,15))
#     rng_x = range(epochs+1)
#     rng_y = range(0,100,5)
#     plt.tight_layout()
#     plt.plot(rng_x,training_accuracies, label="Training" )
#     plt.plot(rng_x,validation_accuracies, label="Validation" )
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.grid()
#     plt.legend()
#     plt.show()

In [16]:
!mkdir models
!mkdir models/pytorch

A subdirectory or file models already exists.
The syntax of the command is incorrect.


In [17]:
import os

# Create a new directory
new_directory = "models/pytorch"
os.mkdir(new_directory)

# Check if the directory was created
if os.path.exists(new_directory):
    print("Directory created successfully!")
else:
    print("Failed to create directory.")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'models/pytorch'

In [ ]:
# # torch.save(model.state_dict(), 'models/pytorch/Pytorch_trial_weights.pt')
# # model = models.resnet50(pretrained=False).to(device)
# # model.fc = nn.Sequential(
# #                nn.Linear(2048, 8).to(device))
# model.load_state_dict(torch.load('models/pytorch/Pytorch_trial_weights.pt'))

In [ ]:
import matplotlib.pyplot as plt

def Plot_Training_Validation_Accuracies(epochs, training_accuracies, validation_accuracies):
    """
    Plots a graph of training and validation accuracies over time.

    Args:
        epochs (list): A list of epoch numbers.
        training_accuracies (list): A list of training accuracies.
        validation_accuracies (list): A list of validation accuracies.
    """

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(19,15))

    # Plot the training and validation accuracies
    ax.plot(epochs, training_accuracies, label="Training")
    ax.plot(epochs, validation_accuracies, label="Validation")

    # Set the labels and grid
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.grid()

    # Add the legend
    ax.legend()

    # Show the plot
    plt.show()


In [ ]:
epochs = list(range(1, 101))

In [ ]:
Plot_Training_Validation_Accuracies(epochs,training_accuracies, validation_accuracies)

In [ ]:

# # Load your dataset
# dataset = ... # Your custom dataset or built-in PyTorch dataset

# # Define split sizes (e.g., 80% train, 10% validation, 10% test)
# train_size = int(0.8 * len(dataset))
# val_size = int(0.1 * len(dataset))
# test_size = len(dataset) - train_size - val_size

# # Split the dataset using random_split
# train_dataset, val_dataset, test_dataset = data.random_split(dataset, [train_size, val_size, test_size])

# # Create dataloaders with appropriate shuffling and sampler options
# train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(model, dataloader, num_classes):
    """
    Plots a confusion matrix for the given PyTorch model and validation data loader.

    Args:
        model (torch.nn.Module): The PyTorch model to evaluate.
        dataloader (torch.utils.data.DataLoader): The validation data loader.
        num_classes (int): The number of classes in the dataset.

    Returns:
        None
    """

    # Set the model to evaluation mode
    model.eval()

    # Initialize lists to store the true and predicted labels
    true_labels = []
    pred_labels = []

    # Iterate over the validation dataset
    with torch.no_grad():
        for images, labels in dataloader:

            # Move the input tensor to the same device as the model weights
            if next(model.parameters()).is_cuda:
                images = images.cuda()

            # Forward pass
            outputs = model(images)

            # Get the predicted labels
            _, predicted = torch.max(outputs.data, 1)


            # Store the true and predicted labels
            true_labels += labels.cpu().numpy().tolist()
            pred_labels += predicted.cpu().numpy().tolist()

    print(true_labels)
    # Compute the confusion matrix
    cm = confusion_matrix(true_labels, pred_labels)
    
    # Plot the confusion matrix
    plt.figure(figsize=(10, 8))
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.imshow(cm, cmap='Blues')
    plt.xticks(range(num_classes),classes_labels)
    plt.yticks(range(num_classes),classes_labels)
    cm = np.around(cm.astype('float')/cm.sum(axis=1)[:,np.newaxis],decimals=2)
    plt.colorbar()
    plt.clim(0, max(cm.max(), cm.sum()))
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      color = "white" if cm[i, j] > thresh else "black"
      plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)


    plt.tight_layout()
    plt.ylim(num_classes, -0.5)
    plt.show()
    

In [ ]:
plot_confusion_matrix(model,dataloaders['validation'],8)